## Problem 1 — Binary Words and Operations

This problem asks us to implement several low-level bitwise functions used in SHA-256.  
They are defined in the **Secure Hash Standard (FIPS 180-4)**.  
All operations must be performed on **32-bit unsigned integers**, so I use `numpy.uint32`.

The functions implemented are:

- Parity(x, y, z)
- Ch(x, y, z)
- Maj(x, y, z)
- Σ0(x), Σ1(x)
- σ0(x), σ1(x)

Each function is introduced in its own section below.


In [5]:
import numpy as np  # For 32-bit unsigned integers (np.uint32)


def rotr(x, n):
    """
    Rotate a 32-bit value x to the right by n bits.
    Used in almost all SHA-256 logical functions.
    """
    x = np.uint32(x)
    n = n % 32
    return np.uint32((x >> n) | (x << (32 - n)))


def shr(x, n):
    """
    Logical right shift of x by n bits.
    Unlike rotation, bits shifted off the right are discarded.
    """
    x = np.uint32(x)
    return np.uint32(x >> n)


## Problem 1A: Parity(x, y, z)

**Goal:** Write a function that returns the XOR of three 32-bit numbers.

From the standard (FIPS 180-4):  
**Parity(x, y, z) = x ⊕ y ⊕ z**

This function is very simple — XOR all three values.  
We ensure everything is cast to `np.uint32` so the result stays 32-bit.


In [6]:
def Parity(x, y, z):

    """Return Parity(x, y, z) = x XOR y XOR z operating on 32-bit words.



    - Inputs may be scalars or numpy arrays; they are cast to np.uint32.

    - The result preserves shape and dtype (np.uint32).

    """

    return u32(u32(x) ^ u32(y) ^ u32(z))

def Parity(x, y, z):
    """
    XOR all three 32-bit words together.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32(x ^ y ^ z)


## Problem 1B: Ch(x, y, z)

**Goal:** Implement the SHA-256 “choose” function.

From the standard:  
**Ch(x, y, z) = (x AND y) XOR (~x AND z)**

Interpretation:  
- If a bit of **x** is 1 → choose bit from **y**  
- If a bit of **x** is 0 → choose bit from **z**  


In [7]:
def Ch(x, y, z):
    """
    SHA-256 choice function:
    For each bit, choose from y if x's bit is 1, else from z.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (~x & z))


## Problem 1C: Maj(x, y, z)

**Goal:** Implement the SHA-256 majority function.

From the standard:  
**Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z)**

Interpretation:  
Bit becomes **1** if *at least two* of x, y, z have a 1 in that position.


In [8]:
def Maj(x, y, z):
    """
    SHA-256 majority function.
    Bit is 1 if two or more of (x, y, z) have bit 1.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


In [9]:
## Problem 2: Fractional Parts of Cube Roots
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [10]:
## Problem 3: Padding
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [11]:
## Problem 4: Hashes
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."

In [12]:
## Problem 5: Passwords 
"""I'll do this section later. Leaving this here to remember the order."""

"I'll do this section later. Leaving this here to remember the order."